# <b><font color="#FF6633">VGG</font></b>

## 包导入与参数定义

In [1]:
# 加载tensorflow模型
import numpy as np
import os
# Uncomment the line below to make GPU unavaliable
# os.environ["CUDA_VISIBLE_DEVICES"] = "-1" 
from tensorflow.keras.layers import Dense,Dropout, Input, concatenate,Flatten
import tensorflow.keras.optimizers as optimizers
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import preprocess_input

## 定义模型
1. 采用**vgg16**进行分类，使用在imageNet上进行预训练的模型进行迁移学习（当然我们的任务和和物体分类差别很大，所有预训练的模型在这里意义不是很大）  
2. 池化选择**平均池化**：因为我们想要的是全局特征，平均池化有利于滤除细微的扰动【但我不确定max_pool是否会更好】   
3. 优化方式为**sgd**,随机性更强的sgd更有利于跳过局部最优，对于我们的任务来说，当然是有必要的  

In [8]:
# 定义宏参数
PICS_WIDTH,PICS_HEIGHT = 512,512
MODEL_LOSS = 'categorical_crossentropy'
MODEL_METRIC = 'categorical_accuracy'
NUM_CATEGS = 10

def InitialiazeModel(head_only,weights,model_name,lr=0.001):
    """
    head_only:选择是否只训练顶端（即自定义的全连接层）
    weights:选择是否从外部导入权重
    model:模型名称
    lr:学习率：默认为0.001
    """
    if model_name == 'VGG19':
        from tensorflow.keras.applications.vgg19 import VGG19
        base_model = VGG19(include_top=False, weights='imagenet',
                      input_shape=(PICS_WIDTH, PICS_HEIGHT, 3), pooling = 'avg')
    # ================================ 该实验选择的分类器 ============================================ #
    elif model_name == 'VGG16':
        from tensorflow.keras.applications.vgg16 import VGG16
        base_model = VGG16(include_top=False, weights='imagenet',
                      input_shape=(PICS_WIDTH, PICS_HEIGHT, 3), pooling = 'avg')
        
    # ============================================================================================== #
    elif model_name == 'MobileNet':
        from tensorflow.keras.applications.mobilenet import MobileNet
        base_model = MobileNet(include_top=False, weights='imagenet',
                      input_shape=(PICS_WIDTH, PICS_HEIGHT, 3), pooling = 'avg')
    elif model_name == 'ResNet50':
        from tensorflow.keras.applications.resnet50 import ResNet50
        base_model = ResNet50(include_top=False, weights='imagenet',
                      input_shape=(PICS_WIDTH, PICS_HEIGHT, 3), pooling = 'avg')
    elif model_name == 'Xception':
        from tensorflow.keras.applications.xception import Xception
        base_model = Xception(include_top=False, weights='imagenet',
                      input_shape=(PICS_WIDTH, PICS_HEIGHT, 3), pooling = 'avg')
    elif model_name == 'InceptionV3':
        from tensorflow.keras.applications.inception_v3 import InceptionV3
        base_model = InceptionV3(include_top=False, weights='imagenet',
                      input_shape=(PICS_WIDTH, PICS_HEIGHT, 3), pooling = 'avg')
    elif model_name == 'InceptionResNetV2':
        from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2
        base_model = InceptionResNetV2(include_top=False, weights='imagenet',
                      input_shape=(PICS_WIDTH, PICS_HEIGHT, 3), pooling = 'avg')
    elif model_name == 'NASNetLarge':
        from tensorflow.keras.applications.nasnet import NASNetLarge
        base_model = NASNetLarge(include_top=False, weights='imagenet',
                      input_shape=(PICS_WIDTH, PICS_HEIGHT, 3), pooling = 'avg')
    elif model_name == 'NASNetMobile':
        from tensorflow.keras.applications.nasnet import NASNetMobile
        base_model = NASNetMobile(include_top=False, weights='imagenet',
                      input_shape=(PICS_WIDTH, PICS_HEIGHT, 3), pooling = 'avg')
    else:
        raise ValueError('Network name is undefined')
    # 是否训练头部    
    if head_only:
        for lay in base_model.layers:
            lay.trainable = False
            
    for i,lay in enumerate(base_model.layers):
            # print(i,lay)
            if i <= 14:
                lay.trainable = False

    # ======================= 全连接层 ======================================
    flat1 = Flatten()(base_model.layers[-1].output)
    # 第一层
    class1 = Dense(256, activation='relu', kernel_initializer='he_uniform')(flat1)
    dropout1 = Dropout(0.2)(class1)
      # 第二层
    class2 = Dense(128, activation='relu', kernel_initializer='he_uniform')(dropout1)
    dropout2 = Dropout(0.2)(class2)
    # 输出层
    output = Dense(NUM_CATEGS, activation='softmax')(dropout2)
    # define new model
    model = Model(inputs=base_model.inputs, outputs=output)
    
    # print(model.summary())
    # 如果存在已有权重则加载已有权重
    if weights != '':
        model.load_weights(weights)
    # ========================= 优化器 =======================================
#     MODEL_OPTIMIZER = optimizers.Adam(lr=0.001)
    MODEL_OPTIMIZER = optimizers.SGD(lr=lr, momentum=0.9, nesterov=True)
    # 编译模型
    model.compile(loss=MODEL_LOSS, optimizer=MODEL_OPTIMIZER, metrics=[MODEL_METRIC])

    return model

In [ ]:
InitialiazeModel(head_only=True,weights='',model_name='VGG16',lr=0.001)

## 开始训练
定义训练用变量

In [10]:
from tensorflow.keras import callbacks
# 模型名
model_name = "VGG16"
train_path = "./datasets/Train"
val_path = "./datasets/Val_main"
BATCH_SIZE = 20
train_sample_num = 9054
val_sample_num = 1555

### 数据生成器
生成类别

In [5]:
category = os.listdir("./datasets/Train")
if '.ipynb_checkpoints' in category:
    category.remove('.ipynb_checkpoints')
print(category)

['Apple_iPhone6Plus', 'Canon_PowerShotA640', 'Huawei_P9', 'Lenovo_P70A', 'Microsoft_Lumia640LTE', 'Nikon_D70s', 'OnePlus_A3003', 'Samsung_GalaxyS5', 'Sony_DSC-W170', 'Xiaomi_RedmiNote3']


很麻烦的一件事是，初始的vgg16权重是在ImageNet上训练的，需要通过preprocess_input函数处理，而问题是keras的生成器函数ImageDataGenerator  
没有提供能进行自定义预处理的接口。一个聪明的方法是**自定义一个生成器封装ImageDataGenerator，对ImageDataGenerator生成的图像进行处理**

In [6]:
def vgg16_gen(directory,classes):
    """
    生成器，对ImageDataGenerator的输出进行处理
    """
    data_gen = ImageDataGenerator()
    train_it = data_gen.flow_from_directory(directory=directory,target_size=(512,512),
                                           classes=category,class_mode= "categorical",
                                            batch_size=BATCH_SIZE)
    while True:
        X,y = next(train_it)
        X = preprocess_input(X)
        yield(X,y)    

--- 
从头开始（毕竟我们数据大）{事实证明这个效果很好}

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping,TensorBoard,ReduceLROnPlateau
from math import ceil

model_name = "VGG16"
train_path = "./datasets/Train"
val_path = "./datasets/Val_main"
BATCH_SIZE = 24
train_sample_num = 30000
val_sample_num = 1555
weights = './model_weight/sgd_not_only_head05.hdf5'
# weights = ''

model = InitialiazeModel(head_only=False,weights=weights,model_name = model_name, lr=0.001)

weights_path_name = "./model_weight/sgd_not_only_head{epoch:02d}.hdf5" 
callbacks = [ModelCheckpoint(weights_path_name, monitor='val_loss', save_best_only=True, verbose=0,
                                             save_weights_only=True),
             EarlyStopping(monitor='val_loss', patience=3, verbose=0.01),
             TensorBoard(log_dir='train_log',update_freq=20000),
             ReduceLROnPlateau(factor=0.5,
                               patience=1, 
                              min_lr=0.0005)]
history1 = model.fit_generator(generator = vgg16_gen(train_path,category),
                    validation_data = vgg16_gen(val_path,category),
                    epochs = 40,
                    steps_per_epoch=ceil(train_sample_num/ BATCH_SIZE),
                    validation_steps=ceil(val_sample_num/ BATCH_SIZE),
                   max_queue_size=20,
                   callbacks=callbacks,
                   verbose = 1)


## 评价
注： 建议在此重启内核

In [ ]:
from models import Vgg16
model = Vgg16(head_only=False,weights='./model_weight/sgd_not_only_head06.hdf5')
BATCH_SIZE = 24
val_sample_num = 1555
val_result = evaluate_generator(vgg16_gen(val_path,category), stepsceil(val_sample_num/ BATCH_SIZE))
